In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from matplotlib.lines import Line2D
import os
import json
import matplotlib.ticker as ticker
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from scipy.interpolate import make_interp_spline, BSpline
os.chdir("C:/Users/Zber/Documents/Dev_program/OpenRadar")
from FER.utils import MapRecord
from sklearn.model_selection import train_test_split

In [3]:

def annotation_update(record_list, width=100, total_frame=300):
    for record in record_list:
        if record.num_frames < width:
            pad = (width - record.num_frames)//2
            if record.onset < pad:
                record.peak += pad*2

            elif (total_frame - record.peak) < pad:
                record.onset -= pad*2
            else:
                record.onset -= pad
                record.peak += pad
        else:
            pad = record.num_frames - width
            record.peak -= pad

        record.path = record.path.replace("Raw_0.bin","{}.npy").replace("\\", "/")

        if record.num_frames != 100:
            record.peak += 1
        assert record.num_frames == 100, 'the num of frames must equal to 100!'
    return record_list


def annotation_attention(record_list, width=30):
    for record in record_list:
        record.onset = math.floor(record.onset * 3 / 10)
        record.peak = record.onset + width - 1
        record.path = record.relative_path.replace("_{}.npy","")
    return record_list


def annotation_append():
    str_arr = []
    str_format = "{} {} {} {} {} {} {} {}"
    npy_path = "{}_{}"
    emotion = 'Neutral'
    subs = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5']

    for sub in subs:
        for i in range(0,30):
            path = (os.path.join(sub, npy_path.format(emotion, i,)) + '_{}.npy').replace("\\", "/")
            label = "0"
            onset = 51
            peak = 150
            offset = -1
            e1 = 0
            e2 = 0
            e3 = 0
            str_arr.append(str_format.format(path, label, onset, peak, offset, e1, e2 , e3))
    return str_arr


def data_split(record_list):
    labels = [r.label for r in record_list]
    train, test = train_test_split(record_list, test_size=0.2, random_state=25, stratify=labels)
    return train, test



# mean and std

In [ ]:
arr = []
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

ele = [-np.inf]

for record in record_list:
    
    npy_path = record.path.format('ele')
    d = np.load(npy_path).flatten()
    mask = np.isin(d, ele)
    if len(d[mask]) >0:
        print(npy_path)
    arr.append(d)

arr = np.concatenate(arr, axis=0)

mean = np.mean(arr)
std = np.std(arr)
print("Mean : {}".format(mean))
print("STD : {}".format(std))


In [ ]:
arr = []
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_og.txt"
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]
train = []

ele = [-np.inf]

for record in record_list:
    
    npy_path = record.path.format('azi')
    d = np.load(npy_path).flatten()
    mask = np.isin(d, ele)
    if len(d[mask]) >0:
        print(npy_path)
    # arr.append(d)
    else:
        train.append(record)

# arr = np.concatenate(arr, axis=0)


str_format = "{} {} {} {} {} {} {} {}"
with open(os.path.join(root_path, "heatmap_annotation_train.txt"), 'a') as f:
    for record in train:
        f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
print("Write {} Records to txt file".format(len(train)))

In [4]:
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Frames"
# annotaton_path = "D:\\Subjects\\annotations_v2.txt"
# annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation.txt"
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
# annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test.txt"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

# new_record_list = annotation_update(record_list)
#
# # str format: path, label, onset, peak, offset, widthError, heightError, indexError
# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation.txt"), 'a') as f:
#     for record in new_record_list:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(new_record_list)))

# train, test = data_split(record_list)

# str format: path, label, onset, peak, offset, widthError, heightError, indexError
# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation_train.txt"), 'a') as f:
#     for record in train:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(train)))
#
# with open(os.path.join(root_path, "heatmap_annotation_test.txt"), 'a') as f:
#     for record in test:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(test)))


# str_arr = annotation_append()
# with open(annotaton_path, 'a') as f:
#     f.writelines('\n'.join(str_arr))
# print("Write {} Records to txt file".format(len(str_arr)))

new_record_list = annotation_attention(record_list)
str_format = "{} {} {} {}\n"
with open(os.path.join(root_path, "annotations_att_train.txt"), 'a') as f:
    for record in new_record_list:
        f.write(str_format.format(record.path, record.onset, record.peak, record.label))
print("Write {} Records to txt file".format(len(new_record_list)))

Write 979 Records to txt file


In [3]:
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

for record in record_list:
    record.path = record.path.replace("C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\","")

str_format = "{} {} {} {} {} {} {} {}"
with open(annotaton_path, 'w') as f:
    for record in train:
        f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
print("Write {} Records to txt file".format(len(train)))

NameError: name 'root_path' is not defined